In [11]:
from pyomo.environ import *
model = AbstractModel()

In [12]:
import pandas as pd
  
# intialise data of lists.
data = {
    'p1':[3,3,3,1,3,2,3,4,4,4,2,2],
    'p2':[4,3,4,2,3,5,7,5,2,3,5,6]
        }
df = pd.DataFrame(data)
df

,p1,p2
0,3,4
1,3,3
2,3,4
3,1,2
4,3,3
5,2,5
6,3,7
7,4,5
8,4,2
9,4,3


In [13]:
model.i = RangeSet(len(df))
model.j = Set(initialize=model.i)
model.t = RangeSet(6)

In [15]:
def initP1(model, i):
    return df.loc[i-1,'p1']

In [16]:
model.P1 = Param(model.i, within=NonNegativeReals, initialize=initP1, mutable=True)

In [17]:
def initP2(model, i):
    return df.loc[i-1,'p2']

In [18]:
model.P2 = Param(model.i, within=NonNegativeReals, initialize=initP2, mutable=True)

In [19]:
model.St_P1 = Var(model.i, initialize=0, bounds=(0,df['p1'].sum()), within=NonNegativeReals)
model.St_P2 = Var(model.i, initialize=0, bounds=(0,df['p1'].sum()+df['p2'].sum()), within=NonNegativeReals)

In [20]:
model.St_OrderP1 = Var(model.i, model.j, initialize=0, bounds=(0,1), within=Binary)

In [21]:
model.Last = Var(initialize=0, bounds=(0,1.5*df['p1'].sum() + df['p2'].sum()), within=NonNegativeReals)

In [22]:
def rule_C1(model, i):
    return model.St_P2[i] - model.St_P1[i] >= model.P1[i]

model.C1 = Constraint(model.i, rule=rule_C1)

In [23]:
def rule_C2(model, i, j):
    if i != j:
        return model.St_P1[j] - model.St_P1[i] >= model.P1[i] - 1.5* df['p1'].sum()*(1 - model.St_OrderP1[i,j])
    else:
        Constraint.Skip
model.C2 = Constraint(model.i, model.j, rule=rule_C2)

In [24]:
def rule_C2B(model, i, j):
    if i != j:
        return model.St_P2[j] - model.St_P2[i] >= model.P2[i] - 1.5* (df['p1'].sum() + df['p2'].sum()) *(1 - model.St_OrderP1[i,j])
    else:
        Constraint.Skip

model.C2B = Constraint(model.i, model.j, rule=rule_C2B)

In [25]:
def rule_C3(model, i,j):
    if i > j:
        return model.St_OrderP1[i,j] + model.St_OrderP1[j,i] == 1
    else:
        return Constraint.Skip

model.C3 = Constraint(model.i, model.j, rule=rule_C3)

In [26]:
def rule_C4(model, i):
    return model.Last >= model.St_P2[i] + model.P2[i]

model.C4 = Constraint(model.i, rule=rule_C4)

In [27]:
def rule_OF(model):
    return model.Last

model.obj1 = Objective(rule=rule_OF, sense=minimize)

In [39]:
opt = SolverFactory('glpk')

In [40]:
opt.solve(model)

RuntimeError: Attempting to solve model=unknown with unconstructed component(s)

In [31]:
model.pprint()

5 Set Declarations
    C2B_index : Size=0, Index=None, Ordered=True
        Not constructed
    C2_index : Size=0, Index=None, Ordered=True
        Not constructed
    C3_index : Size=0, Index=None, Ordered=True
        Not constructed
    St_OrderP1_index : Size=0, Index=None, Ordered=True
        Not constructed
    j : Size=0, Index=None, Ordered=Insertion
        Not constructed

2 RangeSet Declarations
    i : Dimen=1, Size=12, Bounds=(1, 12)
        Key  : Finite : Members
        None :   True :  [1:12]
    t : Dimen=1, Size=6, Bounds=(1, 6)
        Key  : Finite : Members
        None :   True :   [1:6]

2 Param Declarations
    P1 : Size=0, Index=i, Domain=NonNegativeReals, Default=None, Mutable=True
        Not constructed
    P2 : Size=0, Index=i, Domain=NonNegativeReals, Default=None, Mutable=True
        Not constructed

4 Var Declarations
    Last : Size=0, Index=None
        Not constructed
    St_OrderP1 : Size=0, Index=St_OrderP1_index
        Not constructed
    St_P1

RuntimeError: Cannot access '__len__' on AbstractOrderedScalarSet 'j' before it has been constructed (initialized): '__len__' is an attribute on an Abstract component and cannot be accessed until the component has been fully constructed (converted to a Concrete component) using AbstractModel.create_instance() or j.construct().